In [56]:
%load_ext autoreload
%autoreload 2<

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
import pim
import pickle
import numpy as np
import matplotlib.pyplot as plt
import subprocess

from random import uniform
from scipy.optimize import minimize
from pim.models.new.stone import trials


In [58]:
def descend(T_inbound=1500, T_outbound=1500, noise=0.1, 
            phi=(0.1,1.0),beta=(0.0,1.0),T_half=(0.5,5.0),
            epsilon=(1e1,1e5),l=(1e-4,1e-3),c_tot=(1e-8,0.3)):

    bounds = [phi, beta, T_half, epsilon, l, c_tot]
    initial = np.array([uniform(a,b) for a,b in bounds])

    headings, velocities = trials.generate_route(T = 1500, vary_speed = True)

    with open("path.pickle", "wb") as f:
        pickle.dump((headings.tolist(), velocities.tolist()), f)


    def fastbee(x, *args):

        subprocess.run(['../../fastbee/target/release/fastbee', *[str(param) for param in x], str(noise)])

        with open("path.pickle", "rb") as outbound, open("inbound.pickle", "rb") as inbound:
            out_path = pickle.load(outbound)
            in_path = pickle.load(inbound)

        v = out_path[1] + in_path[1]

        position = np.zeros(2)
        path = [position]
        for velocity in v:
            position = position + velocity
            path.append(position)

        closest = min(path[T_outbound:], key = np.linalg.norm)
        return np.linalg.norm(closest)

    result = minimize(fastbee, x0=initial, bounds=bounds)

    return result

In [59]:
res = descend(
    phi=(0.001,0.9),
    beta=(0.01,10.0),
    T_half=(0.1,10.0),
    epsilon=(1e3,1e4),
    l=(1e-5,1e-3),
    c_tot=(1e-2,3e-1),
    noise=0.1
)
print(res)

      fun: 10.207252864407886
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([1.56292362e+09, 6.00514606e+09, 4.64988845e+09, 1.37392490e+09,
       1.85247498e+09, 5.62042794e+09])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 245
      nit: 4
     njev: 35
   status: 0
  success: True
        x: array([4.46604464e-01, 6.13882876e+00, 9.87144854e+00, 5.65649968e+03,
       6.04728477e-04, 2.46410552e-01])


In [60]:
avg = 0
N = 50

if res.success:
    for i in range(N):
        headings, velocities = trials.generate_route(T = 1500, vary_speed = True)

        with open("path.pickle", "wb") as f:
            pickle.dump((headings.tolist(), velocities.tolist()), f)

        subprocess.run(['../../fastbee/target/release/fastbee', *[str(param) for param in res.x], str(0.1)])

        with open("path.pickle", "rb") as outbound, open("inbound.pickle", "rb") as inbound:
            out_path = pickle.load(outbound)
            in_path = pickle.load(inbound)

        v = out_path[1] + in_path[1]

        position = np.zeros(2)
        path = [position]
        for velocity in v:
            position = position + velocity
            path.append(position)

        closest = min(path[1500:], key = np.linalg.norm)
        avg += np.linalg.norm(closest)

    print(avg/N)

124.21390051773136
